In [102]:
%matplotlib inline
from query.datasets.prelude import *
from query.datasets.tvnews.shot_detect import shot_detect, shot_stitch
from scannerpy.stdlib import parsers
from scipy.spatial import distance
from unionfind import unionfind
import cv2

In [45]:
gt = [
    226, 822, 2652, 3893, 4058, 4195, 4326, 4450, 4583, 4766, 5021, 5202, 5294, 5411, 6584,
    7140, 7236, 7388, 7547, 7673, 7823, 7984, 8148, 8338, 8494, 8625, 8914, 9042, 9207, 9308,
    11395, 11823, 12198, 12563, 13516, 13878, 13991, 14162, 14237, 14333, 14488, 14688, 14770,
    14825, 15017, 15537, 15701, 15866, 16012, 16112, 16295, 16452, 16601, 16880, 17018, 17184,
    17310, 17446, 17962, 18713, 18860, 19120, 19395, 19543, 19660, 19839, 19970, 20079, 20248,
    20291, 20862
]
gt = [n - 20 for n in gt]

In [103]:
video = Video.objects.get(path='tvnews/videos/MSNBC_20100827_060000_The_Rachel_Maddow_Show.mp4')
labeler, _ = Labeler.objects.get_or_create(name='shot-histogram')

with Database() as db:
    frame = db.ops.FrameInput()
    histogram = db.ops.Histogram(frame=frame, device=DeviceType.GPU)
    output = db.ops.Output(columns=[histogram])
    job = Job(
        op_args={frame: db.table(video.path).column('frame'),
                 output: video.path + '_hist'})
    bulk_job = BulkJob(output=output, jobs=[job])
    # [hists_table] = db.run(bulk_job, force=True, io_packet_size=10000)
    hists_table = db.table(video.path + '_hist')

    print('Loading histograms...')
    hists = [h for _, h in hists_table.load(['histogram'], parsers.histograms)]
    print('Loaded!')

Loading histograms...
Loaded!


In [104]:
diffs = np.array([
    np.mean([distance.chebyshev(hists[i - 1][j], hists[i][j]) for j in range(3)])
    for i in range(1, len(hists))
])
diffs = np.insert(diffs, 0, 0)

In [100]:
WINDOW_SIZE = 500
GROUP_THRESHOLD = 10
STD_DEV_FACTOR = 1
MAGNITUDE_THRESHOLD = 5000

def compute_shot_boundaries(hists):
    # Compute the mean difference between each pair of adjacent frames
    log.debug('Computing means')
    diffs = np.array([
        np.mean([distance.chebyshev(hists[i - 1][j], hists[i][j]) for j in range(3)])
        for i in range(1, len(hists))
    ])
    diffs = np.insert(diffs, 0, 0)
    n = len(diffs)

    # Do simple outlier detection to find boundaries between shots
    log.debug('Detecting outliers')
    boundaries = []
    for i in range(1, n):
        window = diffs[max(i - WINDOW_SIZE/2, 0):min(i + WINDOW_SIZE/2, n)]
        if diffs[i] > MAGNITUDE_THRESHOLD and diffs[i] - np.mean(window) > STD_DEV_FACTOR * np.std(window):
            boundaries.append(i)
               
    log.debug('Grouping adjacent frames')
    u = unionfind(len(boundaries))
    for i, bi in enumerate(boundaries):
        for j, bj in enumerate(boundaries):
            if abs(bi - bj) < GROUP_THRESHOLD:
                u.unite(i, j)
                break
        
    grouped_boundaries = [boundaries[g[len(g)/2]] for g in u.groups()]
            
    return grouped_boundaries

boundaries = compute_shot_boundaries(hists[:gt[-1]]) 
print('Done!')
print(boundaries)

Computing means
Detecting outliers
Grouping adjacent frames
Done!
[202, 400, 531, 574, 690, 729, 754, 789, 801, 861, 899, 951, 976, 1003, 1022, 1035, 3617, 4418, 4557, 4591, 4761, 4873, 5059, 5959, 6151, 6257, 6341, 6995, 7293, 7786, 8211, 8596, 8668, 9455, 9523, 9837, 10065, 10098, 10182, 11317, 11863, 12510, 13760, 14344, 15441, 16667, 17779, 17953, 18141, 18311, 18506, 18615, 18678, 18737, 19013, 19499, 19612, 19646, 20697]


In [110]:
initial_boundaries = [s.min_frame for s in shot_detect([video])[0]]
final_boundaries = [s.min_frame for s in shot_stitch([video], None, None, None, None)[0]]


In [111]:
local_path = '/tmp/MSNBC_20100827_060000_The_Rachel_Maddow_Show.mp4'
#local_path = '/tmp/MSNBCW_20130404_060000_Hardball_With_Chris_Matthews.mp4'
in_vid = cv2.VideoCapture(local_path)
out_vid = cv2.VideoWriter('test.mkv', cv2.VideoWriter_fourcc(*'XVID'), 30.0, (video.width, video.height))

# offset = 1000
# in_vid.set(cv2.CAP_PROP_POS_FRAMES, offset)

YMAX = 50000
for i in range(20000):
    if i % 50 == 0: print(i)
    ret, frame = in_vid.read()
#     assert ret
    
#     i += offset
    
    wmin = max(i-WINDOW_SIZE/2, 0)
    wmax = i+WINDOW_SIZE/2
    wsize = wmax-wmin
    window = diffs[wmin:wmax]
    mpx = i if i < WINDOW_SIZE/2 else wsize/2
    
    fig = plt.figure(tight_layout=True)
    fig.add_subplot(111)
    
    plt.plot(window, 'tab:blue')

    plt.axvline(x=mpx, color='tab:red')
    plt.axhline(y=np.mean(window) + STD_DEV_FACTOR * np.std(window), color='tab:green')
    plt.axhline(y=MAGNITUDE_THRESHOLD, color='tab:olive')
    
    for b in initial_boundaries:
        idx = b - wmin
        if idx > 0 and idx < wsize:
            plt.scatter(idx, min(diffs[b], YMAX-2000), 
                        c='tab:orange' if b in final_boundaries else 'tab:purple', s=300, marker='X')
    
    plt.ylim(0, YMAX)
    plt.axis('off')
    axes = plt.gca().axes
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)
    fig.canvas.draw()
    
#     plt.show()
#     break
    
    plt.close(fig)
    plt.clf()

    plt_img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    plt_img = plt_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt_img = cv2.resize(plt_img, None, fx=0.5, fy=0.5)
    plt_img = cv2.cvtColor(plt_img, cv2.COLOR_RGB2BGR)
    [ph, pw, pc] = plt_img.shape

    [fh, fw, fc] = frame.shape
    frame[(fh-ph):fh, (fw-pw):fw, :] = plt_img
    
    out_vid.write(frame)
    
out_vid.release()

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

In [85]:
Shot.objects.filter(video=video).delete()
shots = []
for i in range(len(boundaries) - 1):
    start = 0 if i == 0 else boundaries[i]
    end = boundaries[i + 1] - 1
    shots.append(Shot(video=video, labeler=labeler, min_frame=start, max_frame=end))

_ = Shot.objects.bulk_create(shots)

In [97]:
DIST_THRESHOLD = 15
gt_copy = gt[:]

boundaries = [n for n in boundaries if n < gt[-1]]

tp = 0
fp = 0
for i in boundaries:
    valid = None
    for k, j in enumerate(gt_copy):
        if abs(i - j) < DIST_THRESHOLD:
            valid = k
            break
    if valid is None:
        fp += 1
    else:
        tp += 1
        gt_copy = gt_copy[:k] + gt_copy[(k+1):]

fn = len(gt_copy)
print(tp, fp, fn)

precision = tp / float(tp + fp)
recall = tp / float(tp + fn)
print('Precision: {:.3f}, recall: {:.3f}, #det/#gt: {:.3f}'.format(precision, recall, len(boundaries) / float(len(gt))))
print(len(boundaries), len(gt))

print(boundaries)
print(gt_copy)
print(gt)

(59, 36, 12)
Precision: 0.621, recall: 0.831, #det/#gt: 1.338
(95, 71)
[13, 200, 289, 800, 2635, 3872, 4040, 4178, 4306, 4434, 4568, 4749, 5005, 5186, 5268, 5395, 6551, 6701, 7121, 7218, 7369, 7529, 7653, 7804, 7824, 7962, 8130, 8315, 8474, 8575, 8608, 8888, 9019, 9187, 9276, 11348, 11780, 12175, 12238, 12519, 12536, 13084, 13496, 13681, 13818, 13854, 13969, 14009, 14202, 14301, 14424, 14468, 14667, 14721, 14787, 14833, 14981, 15233, 15513, 15608, 15676, 15837, 15969, 16063, 16087, 16270, 16427, 16580, 16650, 16857, 16992, 17056, 17164, 17280, 17409, 17641, 17744, 17939, 18119, 18678, 18791, 19090, 19176, 19365, 19432, 19521, 19643, 19821, 19921, 19940, 20053, 20219, 20252, 20375, 20839]
[11375, 11803, 14142, 14217, 14750, 14805, 14997, 15992, 17426, 18693, 18840, 20271]
[206, 802, 2632, 3873, 4038, 4175, 4306, 4430, 4563, 4746, 5001, 5182, 5274, 5391, 6564, 7120, 7216, 7368, 7527, 7653, 7803, 7964, 8128, 8318, 8474, 8605, 8894, 9022, 9187, 9288, 11375, 11803, 12178, 12543, 13496, 1385